# Descriptor Example: Attribute Validation

## LineItem Take #3: A Simple Descriptor

In [5]:
class Quantity:
    
    def __init__(self, storage_name):
        self.storage_name = storage_name
        
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [9]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [10]:
truffle = LineItem('White truffle', 100, 1)

In [13]:
truffle.__dict__

{'description': 'White truffle', 'price': 1, 'weight': 100}

## LineItem Take #4: Automatic Storage Attribute Names

In [14]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [15]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [16]:
coconuts.weight, coconuts.price

(20, 17.95)

In [17]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [18]:
LineItem.weight

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

In [19]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [20]:
LineItem.price

In [21]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

## LineItem Take #5: A New Descriptor Type

In [1]:
import abc

class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        
class Quantity(Validated):
    """a number greater than zero"""
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    """a string with at least one non-space character"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [3]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

# Overriding Versus Nonoverriding Descriptors

In [27]:
### auxiliary functions for display only ###

def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))
    
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
### essential classes for this example ###

class Overriding:
    """a.k.a. data descriptor or enforded descriptor"""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class OverridingNoGet:
    """an overriding descriptor without ``__get__``"""
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:
    """a.k.a. non-data or shadowable descriptor"""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

## Overriding Descriptor

In [29]:
obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [30]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [31]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [32]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [33]:
obj.__dict__['over'] = 8

In [34]:
vars(obj)

{'over': 8}

In [35]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


## Overriding Descriptor Without __get__

In [36]:
obj.over_no_get

In [37]:
Managed.over_no_get

In [38]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [39]:
obj.over_no_get

In [40]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [41]:
obj.over_no_get = 7
obj.over_no_get

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


9

##  Nonoverriding Descriptor

In [42]:
obj = Managed()
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [43]:
obj.non_over = 7
obj.non_over

7

In [44]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [45]:
del obj.non_over

In [46]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


## Overwriting a Descriptor in the Class

In [47]:
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

# Methods Are Descriptors

In [48]:
obj = Managed()
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x000000A063D6A0F0>>

In [49]:
Managed.spam

<function __main__.Managed.spam>

In [50]:
obj.spam = 7
obj.spam

7

In [51]:
import collections

class Text(collections.UserString):
    
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [52]:
word = Text('forward')
word

Text('forward')

In [53]:
word.reverse()

Text('drawrof')

In [54]:
Text.reverse(Text('backward'))

Text('drawkcab')

In [55]:
type(Text.reverse), type(word.reverse)

(function, method)

In [56]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))

['diaper', (30, 20, 10), Text('desserts')]

In [57]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [58]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse>

In [59]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [60]:
word.reverse.__self__

Text('forward')

In [61]:
word.reverse.__func__ is Text.reverse

True

# Descriptor docstring and Overriding Deletion

In [4]:
help(LineItem.weight)

Help on Quantity in module __main__ object:

class Quantity(Validated)
 |  a number greater than zero
 |  
 |  Method resolution order:
 |      Quantity
 |      Validated
 |      abc.ABC
 |      AutoStorage
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  validate(self, instance, value)
 |      return validated value or raise ValueError
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Validated:
 |  
 |  __set__(self, instance, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from abc.ABC:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------

In [5]:
help(LineItem)

Help on class LineItem in module __main__:

class LineItem(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, description, weight, price)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  subtotal(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  description
 |      a string with at least one non-space character
 |  
 |  price
 |      a number greater than zero
 |  
 |  weight
 |      a number greater than zero

